In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter

In [4]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MNIST dataset 
dataset = torchvision.datasets.MNIST(root='../../data', 
                                     train=True, 
                                     transform=transforms.ToTensor(),  
                                     download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset, 
                                          batch_size=100, 
                                          shuffle=True)

In [5]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size=784, hidden_size=500, num_classes=10):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)  

In [6]:
data_iter = iter(data_loader)
iter_per_epoch = len(data_loader)
total_step = 50000

In [7]:
import matplotlib.pyplot as plt

In [8]:
writer = SummaryWriter()

In [9]:
# Start training
for step in range(total_step):
    
    # Reset the data_iter
    if (step+1) % iter_per_epoch == 0:
        data_iter = iter(data_loader)

    # Fetch images and labels
    images, labels = next(data_iter)
    images, labels = images.view(images.size(0), -1).to(device), labels.to(device)
    
    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Compute accuracy
    _, argmax = torch.max(outputs, 1)
    accuracy = (labels == argmax.squeeze()).float().mean()

    if (step+1) % 100 == 0:
        print ('Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(step+1, total_step, loss.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

        # 1. Log scalar values (scalar summary)
        info = { 'loss': loss.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, step+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), step+1)
            writer.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), step+1)

        # 3. Log training images (image summary)
        info = { 'images': images.view(-1, 1, 28, 28)[:10].cpu() }

        for tag, images in info.items():
            grid = torchvision.utils.make_grid(images)
            writer.add_image(tag, grid, step+1)

Step [100/50000], Loss: 2.2031, Acc: 0.25
Step [200/50000], Loss: 2.0797, Acc: 0.55
Step [300/50000], Loss: 1.9889, Acc: 0.68
Step [400/50000], Loss: 1.8362, Acc: 0.80
Step [500/50000], Loss: 1.6993, Acc: 0.76
Step [600/50000], Loss: 1.6184, Acc: 0.74
Step [700/50000], Loss: 1.4617, Acc: 0.83
Step [800/50000], Loss: 1.3699, Acc: 0.77
Step [900/50000], Loss: 1.2275, Acc: 0.84
Step [1000/50000], Loss: 1.1499, Acc: 0.86
Step [1100/50000], Loss: 1.1403, Acc: 0.82
Step [1200/50000], Loss: 0.9357, Acc: 0.90
Step [1300/50000], Loss: 0.8854, Acc: 0.90
Step [1400/50000], Loss: 0.9040, Acc: 0.87
Step [1500/50000], Loss: 0.9381, Acc: 0.82
Step [1600/50000], Loss: 0.8487, Acc: 0.85
Step [1700/50000], Loss: 0.7131, Acc: 0.88
Step [1800/50000], Loss: 0.7206, Acc: 0.93
Step [1900/50000], Loss: 0.7246, Acc: 0.84
Step [2000/50000], Loss: 0.7244, Acc: 0.85
Step [2100/50000], Loss: 0.6800, Acc: 0.89
Step [2200/50000], Loss: 0.6570, Acc: 0.89
Step [2300/50000], Loss: 0.6300, Acc: 0.87
Step [2400/50000], L